In [ ]:
# IMPORTING LIBRARIES

import pandas as pd
import numpy as np
import requests
from pandas.io.json import json_normalize
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors

In [ ]:
# GETTING THE NEIGHBORHOOD DATA

df = pd.read_html("https://en.wikipedia.org/wiki/List_of_neighborhoods_in_Miami")[0]
miami = df[["Neighborhood","Coordinates"]]
miami.dropna(axis=0,subset=["Coordinates"],inplace=True)
miami.reset_index(drop=True,inplace=True)

In [ ]:
# ADDING LATITUDE AND LONGITUDE COLUMNS

miami["Latitude"] = ""
miami["Longitude"] = ""

In [ ]:
# TRANSFORMING COORDINATES FORMAT

for i in range(len(miami["Neighborhood"])):
    lat = miami["Coordinates"][i][0:6]
    lng = miami["Coordinates"][i][6:13]
    miami.loc[i,"Latitude"] = lat
    miami.loc[i,"Longitude"] = lng

miami = miami[["Neighborhood","Latitude","Longitude"]]

In [27]:
# NEIGHBORHOOD DATA 1 (without venues)

miami.head()

,Neighborhood,Latitude,Longitude
0,Allapattah,25.815,-80.224
1,Arts & Entertainment District,25.799,-80.190
2,Brickell,25.758,-80.193
3,Buena Vista,25.813,-80.192
4,Coconut Grove,25.712,-80.257


In [28]:
# ADDING MORE COLUMNS TO THE DATA FRAME

miami["Police Sations"] = ""
miami["Fire Stations"] = ""
miami["Schools"] = ""
miami["Malls"] = ""
miami["Hospitals"] = ""
miami["Universities"] = ""

In [ ]:
# GETTING THE NUMBER OF VENUES PER NEIGHBORHOOD

client_id = "VLPUTLGCHGYS3FWN44ZSHRRYHKS1MEC0NGBQSXKCD2E2A5OI"
client_secret = "TRCUPBLWIZW45U2RP13IXM4ACGB2X0ZPZKDZFT2CZMEOXSF3"
version = 20200101
radius = 800
limit = 20

categories_id = ["4bf58dd8d48988d12e941735", "4bf58dd8d48988d12c941735", "4bf58dd8d48988d13d941735", "4bf58dd8d48988d1fd941735", "4bf58dd8d48988d196941735", "4bf58dd8d48988d1ae941735"]
categories = ["Police Sations","Fire Stations","Schools","Malls","Hospitals","Universities"]

for i in range(6):
    category_1 = categories_id[i]
    for j in range(len(miami)):
        lat = miami.loc[j,"Latitude"]
        lng = miami.loc[j,"Longitude"]
        url = "https://api.foursquare.com/v2/venues/search?ll={},{}&categoryId={}&client_id={}&client_secret={}&v=20200630&radius={}&limit={}".format(lat,lng,category_1,client_id,client_secret,radius,limit)
        results = requests.get(url).json()
        venues = results['response']['venues']
        venues_df = json_normalize(venues)
        quantity = len(venues_df)
        k = 3+i
        miami.iloc[j,k] = quantity
           
print("Done")

In [30]:
# NEIGHBORHOOD DATA 2 (with venues)

miami.head()

,Neighborhood,Latitude,Longitude,Police Sations,Fire Stations,Schools,Malls,Hospitals,Universities
0,Allapattah,25.815,-80.224,0,1,1,0,1,0
1,Arts & Entertainment District,25.799,-80.190,0,1,1,2,1,1
2,Brickell,25.758,-80.193,0,1,0,2,1,5
3,Buena Vista,25.813,-80.192,2,0,1,4,0,0
4,Coconut Grove,25.712,-80.257,0,0,0,0,0,0


In [ ]:
# CREATING CLUSTERS

miami_dataset = miami[["Police Sations", "Fire Stations", "Schools", "Malls", "Hospitals", "Universities"]]
x = miami_dataset.values[:,1:]
x = np.nan_to_num(x)
cluster_dataset = StandardScaler().fit_transform(x)

k_means = KMeans(init="k-means++", n_clusters=3, n_init=12)
k_means.fit(cluster_dataset)
labels = k_means.labels_
miami["Cluster"] = labels
miami_clusters = miami[["Police Sations", "Fire Stations", "Schools", "Malls", "Hospitals", "Universities", "Cluster"]]
miami_clusters[["Police Sations", "Fire Stations", "Schools", "Malls", "Hospitals", "Universities"]] = miami_clusters[["Police Sations", "Fire Stations", "Schools", "Malls", "Hospitals", "Universities"]].astype(int)
miami_clusters = miami_clusters.groupby("Cluster").mean()

In [32]:
# NEIGHBORHOOD DATA 3 (with venues and clusters)

miami.head()

,Neighborhood,Latitude,Longitude,Police Sations,Fire Stations,Schools,Malls,Hospitals,Universities,Cluster
0,Allapattah,25.815,-80.224,0,1,1,0,1,0,2
1,Arts & Entertainment District,25.799,-80.190,0,1,1,2,1,1,2
2,Brickell,25.758,-80.193,0,1,0,2,1,5,0
3,Buena Vista,25.813,-80.192,2,0,1,4,0,0,2
4,Coconut Grove,25.712,-80.257,0,0,0,0,0,0,1


In [33]:
# CLUSTERS

miami_clusters

,Police Sations,Fire Stations,Schools,Malls,Hospitals,Universities
Cluster,,,,,,
0,4.333333,2.666667,0.666667,3.666667,1.666667,3.333333
1,0.100000,0.000000,0.000000,0.300000,0.000000,0.000000
2,1.727273,0.636364,1.272727,1.727273,1.090909,0.181818


In [34]:
# GETTING MIAMI'S COORDINATES

address = 'Miami, United States' 

geolocator = Nominatim(user_agent = "foursquare_agent")
location = geolocator.geocode(address)
lat = location.latitude
lng = location.longitude
print(lat,lng)

25.7742658 -80.1936589


In [36]:
# CREATING THE MAP

map_clusters = folium.Map(location=[float(lat), float(lng)], zoom_start=12)

# set color scheme for the clusters
x = np.arange(3)
ys = [i + x + (i*x)**2 for i in range(3)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for latitude, longitude, neighborhood, cluster in zip(miami['Latitude'], miami['Longitude'], miami['Neighborhood'], miami['Cluster']):
    label = folium.Popup(str(neighborhood) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [float(latitude), float(longitude)],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [37]:
# GIVING A "WEIGHT" TO EACH VENUE TYPE

poli = 0.2
fire = 0.2
scho = 0.15
mall = 0.05
hosp = 0.3
univ = 0.1

In [38]:
# MULTIPLYING THE WEIGHT BY THE NUMBRE OF VENUES OF EACH TYPE

miami["Police Sations"] = miami["Police Sations"] * poli
miami["Fire Stations"] = miami["Fire Stations"] * fire
miami["Schools"] = miami["Schools"] * scho
miami["Malls"] = miami["Malls"] * mall 
miami["Hospitals"] = miami["Hospitals"] * hosp
miami["Universities"] = miami["Universities"] * univ

In [39]:
# CREATYING A RANKING AND SORTING THE NEIGHBORHOODS

miami["Ranking"] = miami[["Police Sations", "Fire Stations", "Schools", "Malls", "Hospitals", "Universities"]].sum(axis=1)
miami.sort_values(by="Ranking", ascending=False, inplace=True)
miami.reset_index(drop=True,inplace=True)
miami = miami[["Neighborhood","Cluster","Ranking"]]

In [40]:
# NEIGHBORHOOD DATA 4 (with venues, clusters and ranking)

miami.head(10)

,Neighborhood,Cluster,Ranking
0,Downtown,0,3.85
1,Park West,0,2.60
2,Lummus Park,2,2.25
3,Overtown,2,2.10
4,Wynwood,2,1.25
5,Brickell,0,1.10
6,Little Haiti,2,1.05
7,Midtown,2,0.90
8,Arts & Entertainment District,2,0.85
9,West Flagler,2,0.80
